In [ ]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [ ]:
from music21 import *
import numpy as np
import os
import random
from keras.layers import *
from keras.models import *
import keras.backend as K
from keras import metrics
from keras import callbacks

In [ ]:
path = "C:/Users/hibat/Desktop/Github/Music_ai/data/"
files = [i for i in os.listdir(path)]

# Load the dataset and create list of notes 

In [ ]:
def read_midi(file):
    notes=[]
    notes_to_parse = None

  #parsing a midi file
    midi = converter.parse(file)
  #grouping based on different instruments
    s2 = instrument.partitionByInstrument(midi)

  #Looping over all the instruments
    for part in s2.parts:
    #select elements of only piano
        if 'Piano' in str(part): 
            notes_to_parse = part.recurse() 
      #finding whether a particular element is note or a chord
            for element in notes_to_parse:
                print(element)
                if isinstance(element, note.Note):
                      notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                      notes.append('.'.join(str(n) for n in element.normalOrder))
      
    return notes

In [ ]:
all_notes=[]
for i in files:
    all_notes.append(read_midi(os.path.join(path, i)))

In [ ]:
notes = [element for notes in all_notes for element in notes]
len(notes)

In [ ]:
#length of a input sequence
no_of_timesteps = 128      

#no. of unique notes
n_vocab = len(set(notes))  
print(n_vocab)
#all the unique notes
pitch = sorted(set(item for item in notes))  
print(pitch)
#assign unique value to every note
note_to_int = dict((note, number) for number, note in enumerate(pitch))
print(len(note_to_int))

In [ ]:
#preparing input and output sequences
X = []
y = []
for notes in all_notes:
    for i in range(0, len(notes) - no_of_timesteps, 1):
        input_ = notes[i:i + no_of_timesteps]
        #print(input_)
        #print(len(input_))
        output = notes[i + no_of_timesteps]
        #print(output)
        X.append([note_to_int[note] for note in input_])
        y.append(note_to_int[output])

In [ ]:
X = np.reshape(X, (len(X), no_of_timesteps, 1))
#normalizing the inputs
X = X / float(n_vocab)   

In [ ]:
X.shape

In [ ]:
len(y)

# LSTM

 ![title](LSTM.png)

In [ ]:
model = Sequential()
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(128))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',  metrics=['mae', 'acc'])
mc = callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_weights_only=True, period=20)
model.fit(X,np.array(y), epochs=100, batch_size=128)
model.summary()

In [ ]:
model.save('music_ai.h5') # save weights of the model

# Convert output of the model into notes

In [ ]:
def generate_music(model, pitch, no_of_timesteps, pattern):
    
    int_to_note = dict((number, note) for number, note in enumerate(pitch))
    
    
    prediction_output = []
    
    # generate 50 elements
    for note_index in range(50):
        #reshaping array to feed into model
        Input = np.reshape(pattern, (1, len(pattern), 1))
        
        #predict the probability and choose the maximum value
        proba = model.predict(Input)
        
        index = np.argmax(proba)
        
        #convert integer back to the element
        if index > len(int_to_note):
            pass
        else:
            pred = int_to_note[index]
            prediction_output.append(pred)
            
            pattern = list(pattern)
            pattern.append(index/float(n_vocab))
            #leave the first value at index 0
            pattern = pattern[1:len(pattern)]

    return prediction_output

# Convert notes into midi file

In [ ]:
# def convert_to_midi(prediction_output):
    offset = 0
    output_notes = []
    print(output_notes)

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Specify duration between 2 notes
        offset+= 0.5
       # offset += random.uniform(0.5,0.9)

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='music_2.mid')

# Generate music

In [ ]:
# start = np.random.randint(0, len(X)-1)
pattern = X[start]

#load the best model
model=load_model('music_ai.h5')
#generate and save music
music = generate_music(model,pitch,no_of_timesteps,pattern)
convert_to_midi(music)